In [1]:
import os
# Toggling between CPU and GPU
# GPU in use is NVIDIA GEFORCE 940M
# To make the GPU unavailable, uncomment the line of code below
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import time
import sys,humanize,psutil,GPUtil

# Define relevant variables for the ML task
batch_size = 10
num_classes = 10
learning_rate = 0.001
num_epochs = 10

In [3]:
# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
from torch.utils.tensorboard import SummaryWriter
#default log_dir is runs
writer = SummaryWriter('runs/MNIST_Dataset')

In [5]:
#Loading the dataset and preprocessing
train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)


test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [6]:
#Defining the convolutional neural network
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [7]:
model = LeNet5(num_classes).to(device)

#Setting the loss function
cost = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#this is defined to print how many steps are remaining when training
total_step = len(train_loader)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 5, gamma=0.1)

In [8]:
# Define function (For CPU) Memory Report
def cpu_mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))

  # Getting usage of cpu virtual_memory in GB
  print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
  # Getting % usage of cpu virtual_memory
  print('CPU RAM % used: ', psutil.virtual_memory()[2])

In [9]:
# Define function (For GPU) Memory Report
def gpu_mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))

  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))

In [10]:
# Training
t1 = time.time()
for epoch in range(num_epochs): # loop over the dataset multiple times
    total_train_loss = 0.0
    total_test_loss = 0.0

    print ("Epoch: ", epoch)

    model.train()


    total = 0
    for idx, (image, label) in enumerate(train_loader):
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()

        pred = model(image)

        loss = cost(pred, label)
        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        pred = torch.nn.functional.softmax(pred, dim=1)

        for i, p in enumerate(pred):

            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1
  
    total_train_acc = total / len(train_dataset)
    total_train_loss = total_train_loss / (idx + 1)

    total = 0
    model.eval()
    for idx, (image, label) in enumerate(test_loader):
       image, label = image.to(device), label.to(device)

       pred = model(image)
       loss = cost(pred, label)
       total_test_loss += loss.item()
       pred = torch.nn.functional.softmax(pred, dim=1)
       for i, p in enumerate(pred):
          if label[i] == torch.max(p.data, 0)[1]:
              total = total + 1

    total_test_acc = total / len(test_dataset)
    total_test_loss = total_test_loss / (idx + 1)

    if epoch %2 == 0: # every 2'nd epoch
    
        writer.add_scalars('Loss',
                           {'Training': total_train_loss,
                            'Testing': total_test_loss},
                            epoch)
      
        writer.add_scalars('Accuracy',
                          {'Training': total_train_acc,
                           'Testing': total_test_acc},
                           epoch)
        writer.close()
        print("Epoch: ", epoch)

    total_train_loss = 0.0
    total_test_loss = 0.0

  
t2 = time.time() 
print('Finished Training')
print("Training Time: ", t2-t1)
cpu_mem_report()
#gpu_mem_report()

Epoch:  0
Epoch:  0
Epoch:  1
Epoch:  2
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  8
Epoch:  9
Finished Training
Training Time:  877.1922941207886
CPU RAM Free: 3.8 GB
RAM Used (GB): 4.70355968
CPU RAM % used:  55.3


In [11]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
t1 = time.time()  
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
    
t2 = time.time() 
print("Testing Time: ", t2-t1)

Accuracy of the network on the 10000 test images: 98.94 %
Testing Time:  7.207873344421387
